In [1]:
import json
import numpy as np

In [2]:
# inference result texts generated by ./h5loader.ipynb
with open("./model_5_inferResult_bound.txt", "rt") as baseModel_inferText:
    bm_result = list(map(lambda x: json.loads(x), baseModel_inferText.readlines()))
with open("./model_3_inferResult_bound.txt", "rt") as compModel_inferText:
    cm_result = list(map(lambda x: json.loads(x), compModel_inferText.readlines()))
print("Reading inference result text complete")

Reading inference result text complete


In [3]:
# bm_result[a][b][c][d]
# a = [imageEntry_index]
# b = [{0: filepath, 1: inferenceResult}]
# c = [{0: bbox, 1: chance, 2: class}]
# d = [inference_detection_index]
# ''' # toggle comment to use fresh-made/pre-made text file
outputFile = open("./model_comparison.txt", "wt")
count = 0
countNotification = 2000
# if True, then the comparator will count inference detection made in base model and not in comparison model as a total error
# Total error means that the data will be compared to a [0,0,0,0], confidence = 0 inference detection
ignore_processed = False

'''
Inference Comparing Algorithm starts here
'''

# for each entry in base model result:
for bmEntryIndex in range(len(bm_result)): # loop 0
    # bm_resPtr takes constant of two first indexes
    bm_resPtr = bm_result[bmEntryIndex][1] 
    entryDiff_collection = []
    for cmEntryIndex in range(len(cm_result)): # "loop" 1
        # search for corresponding image entry in comparison model result:
        if bm_result[bmEntryIndex][0] == cm_result[(bmEntryIndex + cmEntryIndex) % len(cm_result)][0]:
            # if entry found, then start iterating for each inference detection of base model:
            # cm_resPtr also takes constant of two first indexes
            cm_resPtr = cm_result[(bmEntryIndex + cmEntryIndex) % len(cm_result)][1]
            for bm_inferIndex in range(len(bm_resPtr[0])): # loop 2
                processed = False
                bm_bbox_pred = bm_resPtr[0][bm_inferIndex]
                bm_chance = bm_resPtr[1][bm_inferIndex]
                bm_class = bm_resPtr[2][bm_inferIndex]
                # first, set base value output taking inference at [0,0,0,0] with chance of 0: (total error prep)
                # diff_bbox is the average of absolute differences between bounding box parameter for each detection
                diff_bbox = sum(list(map(lambda x: abs(x), bm_bbox_pred))) / len(bm_bbox_pred)
                # diff_chance is the absolute difference between probability of prediction
                diff_chance = abs(bm_chance)
                # then, search for each inference detection of comparison model 
                if True:
                    # search criteria: bbox parameter that is closest to its base model value
                    # complexity O(2940n^3) - worst case, expected complexity O(2940n^2) due to how data storage is structured
                    for cm_inferIndex in range(len(cm_resPtr[0])): # loop 3
                        # if inferred class is the same AND bbox difference is smaller (compounded for formatting):
                        if cm_resPtr[2][cm_inferIndex] == bm_class:
                            avg_diff = sum(list(map(lambda x, y: abs(x - y), bm_bbox_pred, cm_resPtr[0][cm_inferIndex]))) / len(bm_bbox_pred)
                            if diff_bbox > avg_diff:
                                processed = True
                                diff_bbox = avg_diff
                                diff_chance = abs(bm_chance - cm_resPtr[1][cm_inferIndex])
                else:
                    # assumption of each model is aligned 
                    # i.e. bm_result[a][1][b][x] and cm_result[a][1][b][x] refers to the same inference detection
                    # this may not always be the case, e.g. ./test2017/000000001192.jpg
                    # complexity O(2940n^2) - worst case, expected complexity O(2940n) due to how data storage is structured
                    if bm_inferIndex < len(cm_resPtr[2]) and cm_resPtr[2][bm_inferIndex] == bm_class:
                        processed = True
                        diff_bbox = sum(list(map(lambda x, y: abs(x - y), bm_bbox_pred, cm_resPtr[0][bm_inferIndex]))) / len(bm_bbox_pred)
                        diff_chance = abs(bm_chance - cm_resPtr[1][bm_inferIndex])
                # here, diff_bbox and diff_chance has been fully established for each inference detection
                if ignore_processed or processed:
                    entryDiff_collection.append([diff_bbox, diff_chance])
            # here, entryDiff_collection will be filled for each image entry
            break # breaking loop 1
    # write [bm_result[bmEntryIndex][0], entryDiff_collection] to file
    outputFile.write(json.dumps([bm_result[bmEntryIndex][0], entryDiff_collection]) + "\n")
    count = count + 1
    if (count % countNotification == 0):
        print("# of entry processed:", count)
outputFile.close()
# '''
print("Processing complete -", count, "image entries processed")

# of entry processed: 2000
# of entry processed: 4000
# of entry processed: 6000
# of entry processed: 8000
# of entry processed: 10000
# of entry processed: 12000
# of entry processed: 14000
# of entry processed: 16000
# of entry processed: 18000
# of entry processed: 20000
# of entry processed: 22000
# of entry processed: 24000
# of entry processed: 26000
# of entry processed: 28000
# of entry processed: 30000
# of entry processed: 32000
# of entry processed: 34000
# of entry processed: 36000
# of entry processed: 38000
# of entry processed: 40000
# of entry processed: 42000
# of entry processed: 44000
# of entry processed: 46000
# of entry processed: 48000
# of entry processed: 50000
# of entry processed: 52000
# of entry processed: 54000
# of entry processed: 56000
# of entry processed: 58000
# of entry processed: 60000
# of entry processed: 62000
# of entry processed: 64000
# of entry processed: 66000
# of entry processed: 68000
# of entry processed: 70000
# of entry processed: 72

In [4]:
dataPoints = []
with open("./model_comparison.txt", "rt") as comparisonData:
    for x in list(map(lambda x: json.loads(x), comparisonData.readlines())):
        dataPoints.extend(x[1])
print(len(dataPoints))

250304


In [5]:
bbox_diff_data = list(map(lambda x: x[0], dataPoints))
print("Bounding box parameter absolute difference data")
print("Mean:", np.mean(bbox_diff_data))
print("Standard deviation:", np.std(bbox_diff_data))
print("Maximum:", np.max(bbox_diff_data))
print("50% point (median):", np.median(bbox_diff_data))
print("75% point:", np.quantile(bbox_diff_data, 0.75))
print("95% point:", np.quantile(bbox_diff_data, 0.95))
print("99% point:", np.quantile(bbox_diff_data, 0.99))
print("99,9% point:", np.quantile(bbox_diff_data, 0.999))

Bounding box parameter absolute difference data
Mean: 0.0014982357124932067
Standard deviation: 0.1472459488511145
Maximum: 24.212942123413086
50% point (median): 0.0
75% point: 0.0
95% point: 0.0
99% point: 0.0
99,9% point: 0.0


In [6]:
chance_diff_data = list(map(lambda x: x[1], dataPoints))
print("Probability parameter absolute difference data")
print("Mean:", np.mean(chance_diff_data))
print("Standard deviation:", np.std(chance_diff_data))
print("Maximum:", np.max(chance_diff_data))
print("50% point (median):", np.median(chance_diff_data))
print("75% point:", np.quantile(chance_diff_data, 0.75))
print("95% point:", np.quantile(chance_diff_data, 0.95))
print("99% point:", np.quantile(chance_diff_data, 0.99))
print("99,9% point:", np.quantile(chance_diff_data, 0.999))

Probability parameter absolute difference data
Mean: 1.393058948072135e-05
Standard deviation: 0.001121883747071454
Maximum: 0.20760494470596313
50% point (median): 0.0
75% point: 0.0
95% point: 0.0
99% point: 0.0
99,9% point: 1.5497207641601562e-06
